In [ ]:
!pip install sentence-transformers
# !pip install pytorch
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=153e5fc955bb75c27b87b32f61e1b6b1b02295ff9d6dedd2aba9beb0827d1781
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import pandas as pd, torch
from sentence_transformers import SentenceTransformer,util
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_excel('/content/drive/MyDrive/to_classify_1.xlsx')

# Load the pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Obtain embeddings for the sentences
proposal_embeddings = model.encode(df['proposal_description'].tolist())
topic_embeddings = model.encode(df['iri_category'].tolist())

# Compute and add the semantic similarity to the DataFrame
df['prop_score'] = [(util.pytorch_cos_sim([proposal_embeddings[i]], [topic_embeddings[i]])[0][0].item()+1) for i in range(len(proposal_embeddings))]
df.to_excel('fnl.xlsx')
df.shape

/usr/local/lib/python3.10/dist-packages/sentence_transformers/util.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  a = torch.tensor(a)


(26698, 16)

In [ ]:
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/proposed_topic_keyword_v6.xlsx")
df= df[df['classification_status']==0].head(25000)
df.shape

# Load the sentence transformer model for embeddings
# model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the text data to get the embeddings
embeddings = model.encode(df['proposal_description'].tolist(), show_progress_bar=True)

# Use DBSCAN for density-based clustering
# The `eps` parameter may need to be tuned based on your specific dataset
clustering_model = DBSCAN(eps=0.28, min_samples=32, metric='cosine',n_jobs=-1)
clustering_model.fit(embeddings)

# Add the cluster labels to the DataFrame
df['cluster_id'] = clustering_model.labels_

# Output the DataFrame with the cluster IDs
print(df.head())
df.to_excel('fnl.xlsx')
df.shape

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

                                proposal_description iri_category_id  \
0  01 apr 2022: please note that this is a revisi...             NaN   
1  01 feb 2023: in the situation where the chairm...             NaN   
2  01 feb 2023: please note that shareholders are...             NaN   
3  01 feb 2023: please note that this is a revisi...             NaN   
4  01 jun 2023: please note that this is a revisi...             NaN   

   classification_status iri_category  cluster_id  
0                      0          NaN           0  
1                      0          NaN          -1  
2                      0          NaN          -1  
3                      0          NaN           0  
4                      0          NaN           0  


(25000, 5)

In [ ]:
len(df['cluster_id'].unique())

26

In [ ]:
# Import necessary libraries
from uagents import Agent
import requests
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

class TemperatureAlertAgent(Agent):
    def __init__(self, weather_api_key, email_username, email_password):
        self.weather_api_key = weather_api_key
        self.email_username = email_username
        self.email_password = email_password
        self.user_preferences = {}

    def set_user_preferences(self, user_id, min_temp, max_temp, location, email):
        self.user_preferences[user_id] = {
            'min_temp': min_temp,
            'max_temp': max_temp,
            'location': location,
            'email': email
        }

    def get_weather(self, location):
        # Connect to the OpenWeatherMap API and fetch real-time temperature for the location
        #response = requests.get(f'http://api.openweathermap.org/data/2.5/weather?q={location}&appid={self.weather_api_key}')
        response = requests.get(f"http://api.weatherstack.com/current? access_key = {self.weather_api_key}& query = 'Lucknow'")
        return response.json()['main']['temp']

    def check_temperature(self):
        for user_id, preferences in self.user_preferences.items():
            current_temp = self.get_weather(preferences['location'])
            if current_temp < preferences['min_temp'] or current_temp > preferences['max_temp']:
                self.send_email_alert(user_id)

    def send_email_alert(self, user_id):
        # Send an email alert to the user
        msg = MIMEMultipart()
        msg['From'] = self.email_username
        msg['To'] = self.user_preferences[user_id]['email']
        msg['Subject'] = 'Temperature Alert'
        body = f"The current temperature in {self.user_preferences[user_id]['location']} is outside your preferred range."
        msg.attach(MIMEText(body, 'plain'))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(self.email_username, self.email_password)
        text = msg.as_string()
        server.sendmail(self.email_username, self.user_preferences[user_id]['email'], text)
        server.quit()

TemperatureAlertAgent('7478cb53eebf07c15c1e74073b853a03','formahabharat@gmail.com','gf')

In [ ]:
import os
import zipfile
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define a custom dataset class
class ZipDataset(Dataset):
    def __init__(self, zip_path, tokenizer):
        self.zip_path = zip_path
        self.tokenizer = tokenizer
        self.file_list = self.get_file_list()

    def get_file_list(self):
        with zipfile.ZipFile(self.zip_path, 'r') as zip_ref:
            return zip_ref.namelist()

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        with zipfile.ZipFile(self.zip_path, 'r') as zip_ref:
            with zip_ref.open(self.file_list[idx]) as file:
                text = file.read().decode('utf-8')
        inputs = self.tokenizer.encode(text, add_special_tokens=True)
        return torch.tensor(inputs)

# Set path to the zip file containing the dataset
zip_path = 'dataset.zip'

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Create the custom dataset
dataset = ZipDataset(zip_path, tokenizer)

# Define the batch size and number of training epochs
batch_size = 4
num_epochs = 5

# Create a DataLoader for batching and shuffling the dataset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set the model to training mode
model.train()

# Define the optimizer and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0

    for batch in dataloader:
        batch = batch.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch, labels=batch)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for the epoch
    avg_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

FileNotFoundError: ignored

In [ ]:
FOLDERNAME = None
assert FOLDERNAME is not None

In [ ]:
def fun(x0:float,n:int = 50,y0:float,f:callable) -> List:

In [ ]:
!pip install base
import base.computational_graphs as cg
import base.visualizers as vis

a = cg.Edge(value=5,name='a')
b = cg.Edge(value=3,name='b')
c = cg.Edge(value=2,name='c')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement base (from versions: none)
ERROR: No matching distribution found for base


In [ ]:
import pandas as pd
import numpy as np
from typing_extensions import DefaultDict
a = DefaultDict(lambda:0)
a['p'] = [5,6,7,8]
a['q'] = ['a','b','c','d']
a['r'] = [('a','r'),('b','c'),('d'),(np.nan,5,7,'Hitlerâ€™s')]
a = pd.DataFrame(a)
a

,p,q,r
0,5,a,"(a, r)"
1,6,b,"(b, c)"
2,7,c,d
3,8,d,"(nan, 5, 7, Hitlerâ€™s)"


In [ ]:
badtext = 'Hitlerâ€™s Last Stand'
encoded = badtext.encode('cp1252')
goodtext = encoded.decode('utf-8')
# goodtext = goodtext.decode('utf8')
goodtext

'Hitler’s Last Stand'

In [ ]:
[x for x in a.r[1]]

['b', 'c']

In [ ]:
str(a.r[3][0])=='nan'

True

In [ ]:
if (np.nan in a['r'][3]):
  print(1)

1


In [ ]:
def select_title(row):
  print(row[1])

In [ ]:
a['selected_title'] = a[['q','r']].apply(select_title)

b
('b', 'c')


In [ ]:
l1 = [2,3,'t']
l2 = [('r','t'),6,8]
l1.append(l2)
print(l1)

[2, 3, 't', [('r', 't'), 6, 8]]


In [ ]:
def func():
  return 1
def another_fun(f:callable):
  return f()
another_fun(func)

1

In [ ]:
def calc_h(a: Union[int, float], b: Union[int, float], n: int) -> float:
  h = (b - a)/n
  return h

In [ ]:
import Union from collections
class spyder:
  def _init_(self,sources : Union[Dict[str,str],List[str]], verbose: bool = True,*args: Optional[List], **kwar ) -> None:
    self.sources : Union[Dict[str,str],List[str]] = sources
    self.verbose : bool = verbose
    self.no_links_per_src : int = kwargs.get('no_links_per_src',15)
    self.crawler_type : str = kwargs.get('crawler_type','buildnfetch')


SyntaxError: ignored

1. Python a Interpreter language, which has everything as class. It's Dynamic data type capability and loosely held (doesn't require every fun to return).
2. every class has init -> where we define variables, methods of three types
- @staticmethod - have no relationship with the class, and can be used without instanciating the class. It's a simple fun within the class
- @classmethod - use cls argument and doesn't require instanace of the class to access the method ex. def fun(cls,a,b) within class and use as class.fun(a,b)
- @selfmethod - only callable with the instance of the class ex. def fun(self,a,b) within class and use as obj = class(), obj.fun(a,b)

In [ ]:
import pandas as pd

data = pd.DataFrame({"power_level": [12000, 16000, 4000, 1500, 3000,
                                     2000, 1600, 2000, 300],
                     "uniform color": ["orange", "blue", "black", "orange",
                                       "purple", "green", "orange", "orange","orange"],
                     "species": ["saiyan","saiyan","saiyan","half saiyan",
                                 "namak","human","human","human","human"]},
                     index = ["Goku","Vegeta", "Nappa","Gohan",
                                   "Piccolo","Tien","Yamcha", "Krillin","Roshi"])

data

,power_level,uniform color,species
Goku,12000,orange,saiyan
Vegeta,16000,blue,saiyan
Nappa,4000,black,saiyan
Gohan,1500,orange,half saiyan
Piccolo,3000,purple,namak
Tien,2000,green,human
Yamcha,1600,orange,human
Krillin,2000,orange,human
Roshi,300,orange,human


In [ ]:
# Use .apply() to apply a function to a Series (single column)

def my_function(x, h, l):

    if x >  h:
        return("high")
    else:
      return ([])
    # if x >  l:
    #     return("med")
    # return ("low")


data["power_level"].apply(my_function, args = [10000, 2000])

Goku       high
Vegeta     high
Nappa        []
Gohan        []
Piccolo      []
Tien         []
Yamcha       []
Krillin      []
Roshi        []
Name: power_level, dtype: object

In [ ]:
# Create a convolutional neural network
class Model2(nn.Module):
    """
    Model architecture copying TinyVGG from:
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from?
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*7*7,
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

torch.manual_seed(42)
model_2 = Model2(input_shape=1,
                 hidden_units=10,
                 output_shape=len(class_names)).to(device)
model_2

In [ ]:
import re

def generate_pattern(user_input):
    # Break the input into groups
    groups = re.findall(r'\(([^()]+)\)', user_input)

    # Initialize pattern parts
    positive_lookaheads = []
    negative_lookaheads = []
    include_patterns = []

    # Iterate over each group and process it
    for group in groups:
        # Split the group into alternatives
        alternatives = group.split('|')
        positive_alternatives = []
        negative_alternatives = []

        # Process each alternative
        for alt in alternatives:
            if alt.startswith('!'):
                # It's a negative condition, remove the '!' and create a negative lookahead
                negative_alternatives.append(alt[1:])
            else:
                # It's a positive condition, add it to the positive alternatives
                positive_alternatives.append(alt)

        # If we have positive alternatives, join them with OR operator and word boundaries
        if positive_alternatives:
            include_patterns.append(r'(' + r'|'.join(r'\b' + re.escape(alt) + r'\w*\b' for alt in positive_alternatives) + r')')

        # If we have negative alternatives, add them to negative lookaheads
        if negative_alternatives:
            negative_lookaheads.append(r'(' + r'|'.join(r'\b' + re.escape(alt) + r'\w*\b' for alt in negative_alternatives) + r')')

    # Start building the final pattern
    pattern = r'^'
    # Add negative lookaheads if any
    if negative_lookaheads:
        pattern += r'(?!.*' + r'(?:' + r'|'.join(negative_lookaheads) + r'))'
    # Add the include patterns, separated by any characters
    pattern += r'.*' + r'.*'.join(include_patterns) + r'.*$'

    return pattern

# Example usage:
user_input = "(abc doc|bcd|cde)(def|efg)(!fgh|ghi)(hij|!ijk)(!jkl|!klm)"
pattern = generate_pattern(user_input)
print(pattern)

^(?!.*(?:(\bfgh\w*\b)|(\bijk\w*\b)|(\bjkl\w*\b|\bklm\w*\b))).*(\babc\ doc\w*\b|\bbcd\w*\b|\bcde\w*\b).*(\bdef\w*\b|\befg\w*\b).*(\bghi\w*\b).*(\bhij\w*\b).*$


In [ ]:
^(\babc\ doc\w*|\bbcd\w*|\bcde\w*).*(\bdef\w*|\befg\w*).*(?:(\bghi\w*)(?!f|g|h)).*(?:(\bhij\w*)(?!i|j|k)).*(?:()(?!j|k|l|(|?|:|\|w|*|)|?|k|l|m)).*$



^(?!.*(?:(\bfgh\w*\b)|(\bijk\w*\b)|(\bjkl\w*\b|\bklm\w*\b))).*(\babc\w*\b|\bbcd\w*\b|\bcde\w*\b).*(\bdef\w*\b|\befg\w*\b).*(\bghi\w*\b).*(\bhij\w*\b).*$